In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re
import scipy.optimize as opt
from tqdm import tqdm
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
from sympy import lambdify

In [3]:
#filename = 'pareto_low_adv.csv'
filename = 'pareto_sm.csv'

In [4]:
t_eq=pd.read_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/'+filename)
len(t_eq)

294128

In [5]:
# drop rows that have the same value in column sympy_format.
t_eq=t_eq.drop_duplicates(subset='sympy_format',keep='first')
len(t_eq)

576

In [6]:
t_eq['fitting_format']=''
t_eq['num_fitting_variables']=0
t_eq['initial_constant_guess'] = pd.Series(dtype='object')

number_matching_pattern = r"(?<![a-zA-Z0-9_.])[+-]?(\d+\.\d+|\.\d+|\d+\.|\d+)(?:[eE][-+]?\d+)?"

for row in tqdm(t_eq.iterrows(),total=len(t_eq)):
    #equation = row[1]['sympy_format']
    #equation = parse_expr(equation)

    raw_equation = row[1]['equation']
    # replace the power sign
    raw_equation = raw_equation.replace('^','**')
    equation = str(parse_expr(raw_equation))

    # # swap constants
    # constants = re.findall(number_matching_pattern, equation)


    # variable_list=[]
    # for i in range(len(constants)):
    #     variable_list.append('p['+str(i)+']')


    # for i in range(len(constants)):
    #     equation = equation.replace(constants[i],variable_list[i])

    # swap constants v2
    # Find all unique matches first to avoid duplicates and incorrect indexing
    constants = list(set(re.findall(number_matching_pattern, equation)))
    # Sort constants by their length in descending order to replace longer numbers first, preventing partial replacement issues
    constants.sort(key=len, reverse=True)

    def replace_with_variable(match):
        # Find the matched number in the constants list and get its index
        number = match.group(1)
        index = constants.index(number)

        # Check if the match includes a minus sign
        if match.group(0).startswith('-'):
            sign = '-'
        else:
            sign = ''
        return f'{sign}p[{index}]'

    equation = re.sub(number_matching_pattern, replace_with_variable, equation)

    t_eq.loc[row[0],'fitting_format']=str(equation) 
    t_eq.loc[row[0],'num_fitting_variables']=len(constants)
    t_eq.at[row[0],'initial_constant_guess']=constants

  0%|          | 0/576 [00:00<?, ?it/s]

100%|██████████| 576/576 [00:03<00:00, 160.69it/s]


In [7]:
t_eq=t_eq.drop_duplicates(subset='fitting_format',keep='last')
len(t_eq)

408

In [8]:
#t_eq.to_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/pareto_cleaned.csv',index=False)
t_eq.to_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/'+filename[:-4]+'_cleaned.csv',index=False)

In [9]:
t_eq

,complexity,loss,score,equation,sympy_format,lambda_format,number_constants,variables,number_variables,unique_number_variables,evolutions,iterations,fitting_format,num_fitting_variables,initial_constant_guess
0,1,3.745910,0.000000,x8,x8,PySRFunction(X=>x8),0,{'x8'},1,1,0,0,x8,0,[]
3,7,0.260955,0.052013,((x13 + -2.800066818502902) + (x3 / 0.83308199...,x13 + 1.200362040957336*x3 - 2.800066818502902,PySRFunction(X=>x13 + 1.200362040957336*x3 - 2...,2,"{'x3', 'x13'}",2,2,0,0,x13 + p[1]*x3 - p[0],2,"[2.800066818502902, 1.200362040957336]"
4,9,0.241575,0.038583,((x0 * 1.7877453163832289) + ((x10 * 0.2115519...,1.7877453163832289*x0 - x1 + 0.211551951173565...,PySRFunction(X=>1.7877453163832289*x0 - x1 + 0...,2,"{'x1', 'x10', 'x0'}",3,3,0,0,p[1]*x0 - x1 + p[0]*x10,2,"[0.21155195117356593, 1.7877453163832289]"
5,13,0.240540,0.001074,((x12 - 1.1307340825520393) + (-1.096071482429...,x12 - 1.1307340825520393 - 1.0960714824296827/...,PySRFunction(X=>x12 - 1.1307340825520393 - 1.0...,4,"{'x3', 'x2', 'x12'}",3,3,0,0,x12 - p[2] - p[0]/(x3*(x2 + p[3]) + p[1]),4,"[1.0960714824296827, 0.7258925603600116, 1.130..."
6,17,0.239022,0.001583,(((x12 - 0.17260667647438555) + (-1.0960714824...,-x11/x12 + x12 - 0.17260667647438555 - 1.09607...,PySRFunction(X=>-x11/x12 + x12 - 0.17260667647...,4,"{'x3', 'x11', 'x2', 'x12'}",5,4,0,0,-x11/x12 + x12 - p[0] - p[1]/(p[3]*x2*x3 + p[2]),4,"[0.17260667647438555, 1.0960714824296827, 0.72..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276019,5,0.231833,0.112452,((x0 * 1.6811329193579263) - x8),1.6811329193579263*x0 - x8,PySRFunction(X=>1.6811329193579263*x0 - x8),1,"{'x8', 'x0'}",2,2,4,3487,p[0]*x0 - x8,1,[1.6811329193579263]
276522,21,0.139685,0.008883,(((x0 + -4.917518853110879) - (x10 / ((((x7 + ...,1.475054933608876*x0 - 1.475054933608876*x10/(...,PySRFunction(X=>1.475054933608876*x0 - 1.47505...,3,"{'x10', 'x7', 'x4', 'x0'}",8,4,4,3532,p[1]*x0 - p[1]*x10/(-(x10 - x7)/x4 + x7*(x7 - ...,3,"[0.18146128642111117, 1.475054933608876, 7.253..."
281625,3,0.290302,0.613878,(x0 + -2.5600904505821065),x0 - 2.5600904505821065,PySRFunction(X=>x0 - 2.5600904505821065),1,{'x0'},1,1,4,3958,x0 - p[0],1,[2.5600904505821065]
292788,9,0.206008,0.052086,((x0 * 1.6949829346446144) - (x8 + (x4 / x6))),1.6949829346446144*x0 - x4/x6 - x8,PySRFunction(X=>1.6949829346446144*x0 - x4/x6 ...,1,"{'x6', 'x4', 'x8', 'x0'}",4,4,4,4888,p[0]*x0 - x4/x6 - x8,1,[1.6949829346446144]


In [10]:
t_eq.iloc[5]

complexity                                                                19
loss                                                                0.227855
score                                                               0.023922
equation                   ((((1.1870711720019034 - x1) / ((x13 - 1.18707...
sympy_format               x0 + (1.1870711720019034 - x1)/(x13 - x9/(x9 -...
lambda_format              PySRFunction(X=>x0 + (1.1870711720019034 - x1)...
number_constants                                                           5
variables                                          {'x1', 'x9', 'x13', 'x0'}
number_variables                                                           5
unique_number_variables                                                    4
evolutions                                                                 0
iterations                                                                 0
fitting_format             x0 + (p[1] - x1)/(x13 - x9/(x9 - p[2]) - p[0])...